In [2]:
import sys
import matplotlib.pyplot as plt
import json
import os
import numpy as np
import pandas as pd
import itertools
import cmdstanpy
import jax
from jax.experimental import optimizers
import jax.scipy as jsc
import jax.numpy as jnp
import urllib
import shutil
from sklearn.model_selection import train_test_split
# required install (the rest is preinstalled)
!pip install cmdstanpy==0.9.77
#fast cmdstanpy install
# #(faster than compiling from source via install_cmdstan() function)
# tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
# tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'
# if not os.path.exists(tgz_file):
#     urllib.request.urlretrieve(tgz_url, tgz_file)
#     shutil.unpack_archive(tgz_file)
#import pybrms
from matplotlib import pyplot as plt
sys.path.append('/content/drive/MyDrive/Colab Notebooks/robust_optimization/src')
#######
from generator import sim_y_barThetax, sim_ThetaXy
#from contextPolicy import argmin_lopt_genargmin_lopt_emp, argmin_lpred, argmin_lopt_bar_argmin_lpred
from loss import lopt_NV
#from plot import train_test_err
from sklearn.model_selection import train_test_split

RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. You may be able work around this issue by building jaxlib from source.

In [ ]:
!pip3 install jax

In [ ]:
test_size = 100000
# 1. plot for train, test loss for increasing trainset
for train_size in np.power(2, [i for i in range(3,11)]):
    X, ys, Thetastar = sim_ThetaXy(n_gen= train_size + test_size)
    _, _, y_w, y_w_new = train_test_split(X, ys[0], train_size = train_size)
    X, X_new, y_m, y_m_new = train_test_split(X, ys[1], train_size = train_size)
    train_df, test_df = train_test_err(X, X_new, y_w, y_w_new, y_m, y_m_new, Thetastar)
    plt.plot(train_df['err_w1'], test_df['err_new_w1'], train_df['err_w2'], test_df['err_new_w2'])
    plt.plot(train_df['err_m1'], test_df['err_new_m1'], train_df['err_m2'], test_df['err_new_m2'])

# Approach 1, 2
Main code is in Appedix. 
`argmin_lopt_gen`is plug-in optimizer from Henry's writing,`argmin_lopt_bar_argmin_lpred` is approach1,`argmin_lopt_emp` is approach2. Three optimization engine is used; 
- `optW_lopt_NV.stan` for plug-in optimizer 
- `optTheta_lpred.stan` for approach1 
- $\theta$ grid search within `argmin_lopt_bar_argmin_lpred` for approach2

The last needs to be improved in a more continuous manner.


# Experiment


## Well-specified

In [ ]:
def P_ybarx(theta, x, n_gen = 10):
  return np.random.normal(loc = theta @ x, scale = 1, size = n_gen)
yw = np.array(df.yw)  
df['what1']= argmin_lopt_bar_argmin_lpred(MClass, P_ybarx, yw, X) #MClass = "lin"
df['what2']= argmin_lopt_emp(P_ybarx, yw, X)
profit = 5
cost = 1
print(np.mean([- (profit * min(w, y) - cost * w) for w, y in zip(df.yw, df.what1)]))
print(np.mean([- (profit * min(w, y) - cost * w) for w, y in zip(df.yw, df.what2)]))
# two experiments
#-0.424, -1.706
#-0.327, -1.522

Approach1 is better when well-specified.

## Misspecification type 1: degree

In [ ]:
ym1 = np.array(df.ym1)
df['what1_m']= argmin_lopt_bar_argmin_lpred(MClass, P_ybarx, yw, X) #MClass = "lin"
df['what2_m']= argmin_lopt_emp(P_ybarx, ym1, X)
print(np.mean([-(profit * min(w, y) - cost * w) for w, y in zip(df.ym1, df.what1_m)]))
print(np.mean([-(profit * min(w, y) - cost * w) for w, y in zip(df.ym1, df.what2_m)]))
#3.23, 3.57
#2.95, 3.08

## Misspecification type 2: distribution

In [ ]:
ym2 = np.array(df.ym2)
df['what1_m2']= argmin_lopt_bar_argmin_lpred(MClass, P_ybarx, ym2, X) #MClass = "lin"
df['what2_m2']= argmin_lopt_emp(P_ybarx, ym2, X)
print(np.mean([-(profit * min(w, y) - cost * w) for w, y in zip(df.ym2, df.what1)]))
print(np.mean([-(profit * min(w, y) - cost * w) for w, y in zip(df.ym2, df.what2)]))
# -1.7073
# -1.5114
# approach 1 better so far.

In [ ]:
df

# questions
#### Implementation
## policy.py 
`argmin_l_pred's MClass`
1. Can you give me some example of model class other than linear? 
2. Unlike Adam's previous comment on $y \sim N(\theta' X'X, 1)$ `quad` could be elementwise power $X^2 (n*p)$? (vs $X'X (p*p)$)
3. How to improve grid search of theta in approach2?

#### Experiment design
1.  What setting could make approach 2 outperform for distribution misspecification like it does in degree of predictor misspecification? Is this misspecification type needed?

#### Theory
1. Could approach1 also be thought as empirical risk minization? As the solution depends on the dataset (x_i, y_i) at $\theta_1$ estimation level, I think it could be. Is this is why you used $\hat{w}$, not $w*$, Henry?
2. Is it awkward to express the cost funciton as $l_{opt}$? Thought the input is w, y underneath, it could be implicitly rearranged as function of $\hat{\theta}, \theta$.
3. Does having a feature related to parameteric optimization?
4. $H_\theta := P_\theta()$ is this notation correct?
5. Is Remark2 (P(.) combinatorial/MIP) from SPO paper applicable for approach2? 
$w^{∗}(c) \in \underset{w \in \tilde{S}}{argmin} l_{opt}(w, y)⊆ \underset{w \in S}{argmin} l_{opt}(w, y)$ where S, $\tilde{S}$ are feasible set and its closed convex hull.

# Appendix
## Main code data generator and opt_engine
- generator
- plugin optimizer
- approach 1
- approach 2

much easier to view here: https://github.com/hyunjimoon/robust_optimization/tree/master/src

```
def generator(theta_star, sigma_x, sigma_y, degree, dist, n):
    '''
    Generate data of 1.ground truth (deg=1, norm_dist), 2.(deg>1, norm_dist), 
                     3.(deg = 1, !norm_dist), 3.(deg > 1, !norm_dist)

    Parameters:
        array theta_star: true parameter value
        array sigma_x: array of length p, is the variance of each feature vector dimension, i.e. x_i ~ N(0, sigma_p)
        float sigma_y: noise of outcome sampled as N(y_true, sigma_y)
        int degree: `y_ture|x` str.  #MISSPEC1 degree 1 vs >1
                    y_true = x-linear if degree =1, polynomial degree prop.to amount of model misspecification
        chr dist: `y|y_ture` str. #MISSPEC2 normal_dist vs unif_dist, t_dist, gamma_dist, (TODO mm_dist)
        int n: number of data points to generate
        # x \sim N(0, sigma_x)
        # y|x \sim N(y_true, sigma_y)
        # (x,y) \sim Normal(??)
    Returns:
        np.array X: predictor data of dimension [n, p]
        np.array y: outcome data of dimension [n, 1] = (1,p) * (p, n)
    '''
    # metadata
    p = len(theta_star)
    # Generate predictor: iid MVN with each ith col. predictors share `sigma_x[i]`
    X = np.random.normal(loc = 0, scale = sigma_x, size = [n, p]) # each row is a training point of size p
    def normal_dist(mu,sd, n):
      return np.random.normal(mu, sd, size = n)
    def unif_dist(mu, sd, n):
      return np.random.uniform(mu - np.sqrt(3)*sd, mu + np.sqrt(3)*sd, n)
    if dist == 'unif':
      ys = [P(theta_star @ np.power(X, d).T, np.repeat(sigma_y, n), n) for P in (normal_dist, unif_dist) for d in (1, degree)]
    return X, ys
    
## 2. Policies with given predictors (supervised)
# plug-in opt.solver
def argmin_lopt_gen(P_ybarx, theta, x):
    '''
    Compute optimal solution for each theta and X assuming $Y|X \sim P_{\theta}$
    Parameters:
        function P_ybarx: given x, theta, simulate y vector (should be len(y)>1)
        array theta: model parameter of size [p, 1]
        array x: predictor data of size [1, p] 
        function lopt: loss function, given w(x), y, X, outputs loss
    Returns:
        array w: optimal solution for each x of size [n,1]
    '''
    y_sim = P_ybarx(theta, x, n_gen = 10)
    profit, cost = 5, 1 #todo **kwargs.args()
    data = {'y': list(y_sim), 'n': len(y_sim), 'profit': profit, 'cost': cost} 
    sm = cmdstanpy.CmdStanModel(stan_file="/content/drive/MyDrive/Colab Notebooks/robust_optimization/src/stan/optW_lopt_NV.stan") 
    what = sm.optimize(data).stan_variable('w')
    return what

# approach 1
def argmin_lopt_bar_argmin_lpred(MClass, P_ybarx, y, X, alg_type = "sample"):
    '''
    Compute approach1 optimal solution; separate predict optimize
    Parameters:
         char MClass: model class type specificed with `P_ybarx`
            "lin": y_true = theta' * X
            "quad": y_true = theta' * X^2
          np.array y: outcome data of size [n, 1]
          np.array X: predictor data of size [n, p] 
          chr alg_type: Solver algorithm
    Returns:
        np.array what: predict then optimize optimal solution of size [n,1]
    '''
    thetahat1 = argmin_lpred(MClass, y, X, alg_type = alg_type)
    what = np.repeat(np.nan, len(X))
    for i in range(len(X)):
        what[i] = argmin_lopt_gen(P_ybarx, thetahat1, X[i])
    return what

# approach 2
def argmin_lopt_emp(P_ybarx, y, X):
    '''
    Compute approach2 optimal solution; empirical objective minimization
    Parameters:
        function P_ybarx: simulator function given X and theta, outputs y
        array y: outcome data of size [n, 1]
        array X: predictor data of size [n, p] 
    Returns:
        array what: optimal solution for each x of size [n,1]
    '''
    # DO NOT CONFUSE THE TWO
    # what_x (\hat{w}(\theta, x_i))
    # what (\hat{w}(\hat{\theta_2}, x))
    theta1 = np.arange(1, 3, 0.5)
    theta2 = np.arange(1, 3, 0.5)
    theta1, theta2 = np.meshgrid(theta1, theta2)
    profit, cost = 5, 1
    M = 1000000
    for theta in zip(theta1.ravel(), theta2.ravel()):
      loss = 0
      for i in range(len(X)):
        what_x = argmin_lopt_gen(P_ybarx, theta, X[i])
        loss -= (profit * min(what_x, y[i]) - cost * what_x)
        #lopt_NV(what, y[i], profit, cost) 'numpy.float64' object cannot be interpreted as an integer
      if loss < M:
        thetahat2 = theta
    what = np.repeat(np.nan, len(X))
    for i in range(len(X)):
        what[i] = argmin_lopt_gen(P_ybarx, thetahat2, X[i])
    return what

def argmin_lpred(MClass, y, X, alg_type = "sample"):
    '''
    Solve parameter that best predicts y given X assuming $y\sim N(X * theta, 1)$
    Parameters:
        char MClass: model class type specificed with `P_ybarx` which is user-defined 
             input for `argmin_lopt_gen`, `argmin_lopt_emp` 
             SHOULD comply with "P_ybarx_f{MClass}.stan"
             "lin": y_true = theta' * X
             "quad": y_true = theta' * X^2
        np.array y: outcome data of size [n, 1]
        np.array X: predictor data of size [n, p] 
        chr alg_type: Solver algorithm
             "sample": MAP with improper uniform priors (undeclared) in stan file with HMC
             "optimize": Maximum likelihood estimate () with "lbfgs", "bfgs", "newton"  
             "varaiational": variational inference with ADVI, RVI
             https://mc-stan.org/cmdstanpy/examples/Maximum%20Likelihood%20Estimation.html
             https://mc-stan.org/cmdstanpy/examples/Variational%20Inference.html
    Returns:
        np.array thetahat: optimal parameter value of size [p, 1]
    '''
    data = {'X':X.tolist(), 'y':y, 'n': len(X), 'p': len(X[1])} 
    sm = cmdstanpy.CmdStanModel(stan_file="/content/drive/MyDrive/Colab Notebooks/robust_optimization/src/stan/optTheta_lpred.stan") # true theta: 2, 2
    if alg_type == "sample":
      thetahat = np.mean(sm.sample(data).stan_variable('beta'), axis =0) # 2.00, 1.98
    if alg_type == "optimize":
      thetahat = sm.optimize(data).stan_variable('beta') # 2.00 , 1.98
    if alg_type == "variational": 
      thetahat = np.mean(sm.sample(data).stan_variable('beta'), axis =0) # 1.99, 1.99
    return thetahat

```

EOD 0906.